## Demos on Chinese Restaurant Process
#### compare theoretical and emprical results for $E(K)$ and $P(K=k)$
## 中国餐馆过程的演示

Prof Richard Xu, 徐亦达教授

University of Technology Sydney (UTS) 悉尼科技大学

Yida.Xu@uts.edu.au

2018-05-01

#### this is my implementation of unsigned Stirling number of the first kind using the following relations:
##### 这是用递归的方式写的第一种Stirling值的取得方法
$s(n + 1, k) = n s(n, k) + s(n, k - 1)$

be aware that this can be quite slow when $n$ is large

In [8]:
def stirling(n,k):

    if n<=0:
        return 1
     
    elif k<=0:
        return 0
     
    elif (n == k):
        return 1
     
    elif n!=0 and n==k:
        return 1
     
    elif k >n:
        return 0
 
    else:
        return (n-1) * stirling(n-1,k)+ stirling(n-1,k-1)

this is to test to see if "unsigned Stirling number of first kind" works


In [9]:
for n in range(1,5):
    for k in range(1,5):
        print "str(", n, ",", k, ")=", stirling(n,k)
    print "\n"

str( 1 , 1 )= 1
str( 1 , 2 )= 0
str( 1 , 3 )= 0
str( 1 , 4 )= 0


str( 2 , 1 )= 1
str( 2 , 2 )= 1
str( 2 , 3 )= 0
str( 2 , 4 )= 0


str( 3 , 1 )= 2
str( 3 , 2 )= 3
str( 3 , 3 )= 1
str( 3 , 4 )= 0


str( 4 , 1 )= 6
str( 4 , 2 )= 11
str( 4 , 3 )= 6
str( 4 , 4 )= 1




The function below samples from the joint distribution **directly** $\Pr(z_1, z_2, \dots z_N | \alpha)$
by using $(z_1, z_2,\dots, z_n) \sim \big( \Pr(z_1)\Pr(z_2|z_1)\dots \Pr(z_n | z_1, \dots z_{n-1}) \equiv \Pr(z_n, z_1, \dots z_{n}) \big)$
这个函数是直接从联合分布里采样

In [10]:
import numpy as np

def Draw_CRP_Direct_Sample(N = 10, alpha = 3, T = 50, VERBOSE = False):

    Z_table = np.zeros((T,N))
    
    for t in range(T):
    
        Z = np.zeros(N,dtype=int)

        for i in range(N):

            if i == 0:
                Z[i] = 1
            else:
                if VERBOSE:
                    print Z
                unique, counts = np.unique(Z, return_counts=True)

                # remove the zeros unsigned tables
                if unique[0] == 0:
                    unique = np.delete(unique,0)
                    counts = np.delete(counts,0)

                #if VERBOSE:
                #    print "unique,counts,alpha", unique,counts,alpha

                # added alpha to the end of the counts (weights) array
                counts = np.append(counts,alpha)

                # also the new table index will be the max of table seen so far
                unique = np.append(unique,max(unique)+1)

                #print "np.append(counts,alpha)",counts

                #if VERBOSE:
                #    print sum(counts)
                u = np.random.uniform()*sum(counts)

                a_counts = np.cumsum(counts)

                if VERBOSE:
                    print counts, u, a_counts > u

                # first index where accumuated sum is greater than random variable
                index =  np.argmax(a_counts > u)

                #print "index", index

                Z[i] = unique[index]

            if VERBOSE:
                print Z
                print("\n\n") 
        
        
        Z_table[t,:] = Z
    return Z_table

This function below samples from the joint distribution $\Pr(z_1, z_2, \dots z_N | \alpha)$ using **Gibbs sampling**
这个函数用吉布斯采样从联合分布$\Pr(z_1, z_2, \dots z_N | \alpha)$中采样

In [11]:
import numpy as np

def Draw_CRP_Gibbs_Sample(N = 10, alpha = 3, T = 50, burn_in = 10, VERBOSE = False):
    
    Z = np.ones(N,dtype=int)
    Z_table = np.zeros((T,N))

    for t in range(T+burn_in):

        for i in range(N):

            if VERBOSE:
                print Z

            # remove current table assignment
            Z[i] = 0

            if VERBOSE:
                print Z


            unique, counts = np.unique(Z, return_counts=True)

            # remove the zeros in unassigned tables
            if unique[0] == 0:
                unique = np.delete(unique,0)
                counts = np.delete(counts,0)

            if VERBOSE:
                print "unique,counts,alpha", unique,counts,alpha

            # added alpha to the end of the counts (weights) array
            counts = np.append(counts,alpha)

            # also the new table index will be the max of table seen so far
            unique = np.append(unique,max(unique)+1)

            #print "np.append(counts,alpha)",counts

            #if VERBOSE:
            #    print sum(counts)
            u = np.random.uniform()*sum(counts)

            a_counts = np.cumsum(counts)

            if VERBOSE:
                print counts, u, a_counts > u

            # first index where accumuated sum is greater than random variable
            index =  np.argmax(a_counts > u)

            #print "index", index

            Z[i] = unique[index]

            if VERBOSE:
                print Z
                print("\n") 

        old_table = np.unique(Z)
        new_table = np.array(range(1,len(old_table)+1))

        for k in range(len(old_table)):
            Z[Z == old_table[k]]=new_table[k]

        if t >= burn_in:
            Z_table[t-burn_in,:] = Z

        if VERBOSE:
            print Z
            print("\n\n\n") 

    if VERBOSE:
        print Z_table
    
    return Z_table


    



#### in the following, we compare sample $\widehat{E}(K)$ vs true $E(K)$, and sample $\widehat{P}(K = k)$ vs true$P(K= k)$
$K$ is the number of tables

在以下的函数里，我们对$K$的样本期望值和$K$理论期望值进行比较; 我们还对$P(K=k)$的样本值和$P(K=k)$理论期望值进行比较; 

In [12]:
from scipy.special import digamma, gamma, gammaln
from math import log,exp

alpha = 1
N = 5
T = 10000
VERBOSE = False
burn_in = int(T/10)

# ------------------------------------------
# call either of the following function
# 您可以选择运行下面的其中一个函数
# ------------------------------------------

Z_table = Draw_CRP_Direct_Sample(N, alpha , T , VERBOSE)

#Z_table = Draw_CRP_Gibbs_Sample(N , alpha , T , burn_in , VERBOSE)




table_numbers = np.zeros(T,dtype=int)

for t in range(T):
    unique, count = np.unique(Z_table[t,:], return_counts=True)
    table_numbers[t] = len(count)


# ------------------------------------------
# we compare sample E(K) vs true E(K)
# 我们对 K 的样本期望值和理论期望值进行比较
# ------------------------------------------   

exp_average = np.mean(table_numbers)    
theory_average = alpha * (digamma(alpha + N) - digamma(alpha))
print "sample E(K) = ", exp_average, " theorical E(K) = ", theory_average
print "\n"

# ------------------------------------------
# We compared Pr(K = k) between sample and true
# 我们对 Pr(K=k) 的样本期望值和理论期望值进行比较
# ------------------------------------------   

unique, count = np.unique(table_numbers, return_counts=True)

for t in range(len(unique)):
    k = unique[t]
    exp_prob = count[t]* 1.000 / T
    
    # ------------------------------------------
    # to avoid overflow, we use a little trick: 
    # 为防止变量接近0，或变成极值，我们会用以下的小技巧:
    # a*b/c = exp(log(a)+log(b)-log(c))
    # ------------------------------------------   
    
    theory_prob = log(stirling(N,k))+k*log(alpha)+gammaln(alpha)- gammaln(alpha + N)
    theory_prob = exp(theory_prob)
    
    print "sample Pr(K = ", unique[t],") = ", exp_prob, "; theorical Pr(K = ", unique[t],") = ", theory_prob

sample E(K) =  2.2741  theorical E(K) =  2.2833333333333337


sample Pr(K =  1 ) =  0.1991 ; theorical Pr(K =  1 ) =  0.2
sample Pr(K =  2 ) =  0.4252 ; theorical Pr(K =  2 ) =  0.416666666667
sample Pr(K =  3 ) =  0.2859 ; theorical Pr(K =  3 ) =  0.291666666667
sample Pr(K =  4 ) =  0.0821 ; theorical Pr(K =  4 ) =  0.0833333333333
sample Pr(K =  5 ) =  0.0077 ; theorical Pr(K =  5 ) =  0.00833333333333
